In [1]:
import tweepy

In [35]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json 
import os
import tweepy

import pandas as pd
from dotenv import load_dotenv
load_dotenv()

TWITTER_API_KEY = os.getenv("TWITTER_API_KEY")
TWITTER_API_SECRET = os.getenv("TWITTER_API_SECRET")
TWITTER_ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
TWITTER_ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

auth = tweepy.OAuthHandler(TWITTER_API_KEY, TWITTER_API_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [3]:
screen_name='LambdaSchool'

In [4]:
def get_information(screen_name):
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,tweet_mode='extended')
    tweets=[]
    retweet_count=[]
    favorite_count=[]
    for tweet in range(len(new_tweets)):

        status = new_tweets[tweet]

        #convert to string
        json_str = json.dumps(status._json)

        #deserialise string into python object
        parsed = json.loads(json_str)
        tweets.append(parsed.get('full_text'))
        retweet_count.append(parsed.get('retweet_count'))
        favorite_count.append(parsed.get('retweet_count'))
    #return a dataframe
    return  pd.DataFrame(list(zip(tweets,retweet_count,favorite_count)), 
               columns =['tweets','retweet_count','favorite_count']) 
    
    
    

In [5]:
df =get_information(screen_name)

In [6]:
# get the user tweet post where followers most engaged with

df=df.sort_values(by=['retweet_count', 'favorite_count'],ascending=False)

In [8]:

df['tweets_clean'] = df['tweets'].apply(lambda x: x[0:-1].replace('\n\n', ' '))

In [9]:
df['tweets_clean']

126    RT @stabby_lambda: My wife was just laid off b...
153    RT @WomenWhoCode: 👩🏾‍💻Grace Hopper was one of ...
118    RT @Austen: Lambda School students turn out do...
179    RT @swoleengineer: I have 2 payments left on m...
191    RT @Austen: There’s been some confusion recent...
61     RT @Austen: Look at the #covid19 tracker Lambd...
162    RT @Austen: Lambda School’s ISA is capped. It ...
79     RT @swoleengineer: As of today, my @Lambdascho...
29     RT @Austen: Nothing makes me happier than when...
140    RT @Austen: What it’s like to finally show you...
83     School closures due to #COVID19 have made #rem...
156    RT @Austen: Lambda School has been perfecting ...
100    RT @reallycoolcoder: I started @LambdaSchool e...
109    In light of #COVID19, we see many companies st...
189    RT @Austen: Such a weird rumor: Students are n...
193    RT @Austen: A couple years ago youtuber Josh F...
115    RT @Austen: Income share agreements are an inc...
37     RT @TheBigKnell: Excited

In [10]:


import re

def clear_emoji(text):
    emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [11]:
df['tweets_clean']=df['tweets_clean'].apply(clear_emoji)

In [12]:

df['tweets_hashtag']=df['tweets_clean'].apply(lambda x:[col for col in x.split() if col.startswith('#')])

In [13]:
df['tweets_hashtag'] = df['tweets_hashtag'].apply(lambda x: ','.join(map(str, x)))

In [14]:
df['tweets_mention']=df['tweets_clean'].apply(lambda x:[col for col in x.split() if col.startswith('@')])

In [15]:
df['tweets_mention'] = df['tweets_mention'].apply(lambda x: ','.join(map(str, x)))

In [16]:
df['tweets']=df['tweets_clean']

In [17]:
df=df.drop('tweets_clean', axis=1)

In [18]:
df

,tweets,retweet_count,favorite_count,tweets_hashtag,tweets_mention
126,RT @stabby_lambda: My wife was just laid off b...,2742,2742,,@stabby_lambda:
153,RT @WomenWhoCode: ‍Grace Hopper was one of the...,243,243,,@WomenWhoCode:
118,RT @Austen: Lambda School students turn out do...,188,188,#COVID19,@Austen:
179,RT @swoleengineer: I have 2 payments left on m...,162,162,,"@swoleengineer:,@LambdaSchool"
191,RT @Austen: There’s been some confusion recent...,101,101,,@Austen:
61,RT @Austen: Look at the #covid19 tracker Lambd...,66,66,#covid19,@Austen:
162,RT @Austen: Lambda School’s ISA is capped. It ...,64,64,,@Austen:
79,"RT @swoleengineer: As of today, my @Lambdascho...",53,53,,"@swoleengineer:,@Lambdaschool"
29,RT @Austen: Nothing makes me happier than when...,45,45,,@Austen:
140,RT @Austen: What it’s like to finally show you...,42,42,,@Austen:


In [19]:
import numpy as np
import gensim

In [20]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

import pandas as pd

In [21]:
import spacy

In [22]:

from spacy.tokenizer import Tokenizer

In [23]:
nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [24]:
STOP_WORDS = nlp.Defaults.stop_words.union(["&amp","rt","lambda","&amp;","i’m","we're","you’re","it’s",'thanks','student','school.'])

In [25]:
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(df['tweets'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        if (token.text.lower() not in STOP_WORDS) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [26]:
id2word = corpora.Dictionary(df['tokens'])

In [27]:

corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [28]:

lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [29]:
lda.print_topics()

[(0,
  '0.009*"celebrating" + 0.006*"learn" + 0.006*"resumé" + 0.006*"working" + 0.006*"curriculum" + 0.006*"school\'s" + 0.006*"weeks" + 0.006*"know" + 0.006*"shares" + 0.006*"alums\'"'),
 (1,
  '0.015*"#womenoflambda" + 0.011*"school" + 0.011*"meet" + 0.009*"help" + 0.009*"students" + 0.007*"got" + 0.007*"#tech" + 0.007*"#seeyourselfatlambda" + 0.004*"coordinator" + 0.004*"jashele"'),
 (2,
  '0.005*"world" + 0.005*"school" + 0.005*"wanted" + 0.005*"got" + 0.005*"amazing" + 0.005*"@theblorble:" + 0.005*"year" + 0.005*"hired" + 0.005*"surreal," + 0.005*"it\'s"'),
 (3,
  '0.009*"brit" + 0.006*"@austen:" + 0.006*"students" + 0.006*"help" + 0.006*"need" + 0.006*"write" + 0.006*"source" + 0.006*"interview" + 0.006*"code" + 0.006*"passionate"'),
 (4,
  '0.010*"school" + 0.008*"people" + 0.008*"@lambdaschool" + 0.008*"years" + 0.005*"team" + 0.005*"lambda," + 0.005*"staff" + 0.005*"know" + 0.005*"place" + 0.005*"experience"'),
 (5,
  '0.010*"reported" + 0.010*"salary" + 0.007*"school" + 0.00

In [30]:

words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [31]:

topics = [' '.join(t[0:5]) for t in words]

In [32]:

for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
celebrating learn resumé working curriculum

------ Topic 1 ------
#womenoflambda school meet help students

------ Topic 2 ------
world school wanted got amazing

------ Topic 3 ------
brit @austen: students help need

------ Topic 4 ------
school people @lambdaschool years team

------ Topic 5 ------
reported salary school 2 python

------ Topic 6 ------
school students #womenoflambda incorrect hey

------ Topic 7 ------
weeks ‍‍ students #seeyourselfatlambda computer

------ Topic 8 ------
students virtual #womenoflambda emily today

------ Topic 9 ------
#ds apply help #datascience data

------ Topic 10 ------
report outcomes 2019 web new

------ Topic 11 ------
school students @austen: software #covid19

------ Topic 12 ------
thank end school today, safe

------ Topic 13 ------
@lambdaschool experience school job jane

------ Topic 14 ------
community school kaitlyn it. time

